# Įvadas į tikimybių teoriją ir statistiką
Šiame užrašų knygelėje žaisime su kai kuriomis anksčiau aptartomis sąvokomis. Daugelis tikimybių ir statistikos sąvokų yra gerai atspindėtos pagrindinėse duomenų apdorojimo bibliotekose Python kalboje, tokiose kaip `numpy` ir `pandas`.


In [ ]:
import numpy as np
import pandas as pd
import random
import matplotlib.pyplot as plt

## Atsitiktiniai dydžiai ir pasiskirstymai
Pradėkime nuo 30 reikšmių mėginio paėmimo iš tolydaus pasiskirstymo nuo 0 iki 9. Taip pat apskaičiuosime vidurkį ir dispersiją.


In [ ]:
sample = [ random.randint(0,10) for _ in range(30) ]
print(f"Sample: {sample}")
print(f"Mean = {np.mean(sample)}")
print(f"Variance = {np.var(sample)}")

Norėdami vizualiai įvertinti, kiek skirtingų reikšmių yra mėginyje, galime nubrėžti **histogramą**:


In [ ]:
plt.hist(sample)
plt.show()

## Realių duomenų analizė

Vidurkis ir dispersija yra labai svarbūs analizuojant realius duomenis. Įkelkime duomenis apie beisbolo žaidėjus iš [SOCR MLB Height/Weight Data](http://wiki.stat.ucla.edu/socr/index.php/SOCR_Data_MLB_HeightsWeights)


In [ ]:
df = pd.read_csv("../../data/SOCR_MLB.tsv",sep='\t', header=None, names=['Name','Team','Role','Weight','Height','Age'])
df


> Čia mes naudojame paketą, vadinamą [**Pandas**](https://pandas.pydata.org/), duomenų analizei. Vėliau šiame kurse daugiau kalbėsime apie Pandas ir darbą su duomenimis Python kalboje.

Apskaičiuokime vidutines amžiaus, ūgio ir svorio reikšmes:


In [ ]:
df[['Age','Height','Weight']].mean()

Dabar susitelkime į ūgį ir apskaičiuokime standartinį nuokrypį bei dispersiją:


In [ ]:
print(list(df['Height'])[:20])

In [ ]:
mean = df['Height'].mean()
var = df['Height'].var()
std = df['Height'].std()
print(f"Mean = {mean}\nVariance = {var}\nStandard Deviation = {std}")

Be vidurkio, prasminga pažvelgti į medianą ir kvantilius. Juos galima vizualizuoti naudojant **dėžės diagramą**:


In [ ]:
plt.figure(figsize=(10,2))
plt.boxplot(df['Height'].ffill(), vert=False, showmeans=True)
plt.grid(color='gray', linestyle='dotted')
plt.tight_layout()
plt.show()

Taip pat galime sudaryti dėžutės diagramas mūsų duomenų rinkinio dalims, pavyzdžiui, suskirstytoms pagal žaidėjo vaidmenį.


In [ ]:
df.boxplot(column='Height', by='Role', figsize=(10,8))
plt.xticks(rotation='vertical')
plt.tight_layout()
plt.show()

> **Pastaba**: Ši diagrama rodo, kad vidutiniškai pirmųjų bazės žaidėjų ūgiai yra didesni už antrųjų bazės žaidėjų ūgius. Vėliau sužinosime, kaip galime formaliau patikrinti šią hipotezę ir kaip parodyti, kad mūsų duomenys statistiškai reikšmingi tai parodyti.  

Amžius, ūgis ir svoris yra tęstiniai atsitiktiniai kintamieji. Kokia, jūsų manymu, yra jų pasiskirstymo forma? Geras būdas sužinoti – nubraižyti reikšmių histogramą: 


In [ ]:
df['Weight'].hist(bins=15, figsize=(10,6))
plt.suptitle('Weight distribution of MLB Players')
plt.xlabel('Weight')
plt.ylabel('Count')
plt.tight_layout()
plt.show()

## Normalus pasiskirstymas

Sukursime dirbtinį svorių pavyzdį, kuris seka normalų pasiskirstymą su tokiu pačiu vidurkiu ir dispersija kaip mūsų realūs duomenys:


In [ ]:
generated = np.random.normal(mean, std, 1000)
generated[:20]

In [ ]:
plt.figure(figsize=(10,6))
plt.hist(generated, bins=15)
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(10,6))
plt.hist(np.random.normal(0,1,50000), bins=300)
plt.tight_layout()
plt.show()

Kadangi dauguma tikrų gyvenimo reikšmių yra normaliai pasiskirsčiusios, neturėtume naudoti tolydaus atsitiktinių skaičių generatoriaus imčių duomenims generuoti. Štai kas nutinka, jei bandysime generuoti svorius naudojant tolydų pasiskirstymą (generuotą su `np.random.rand`):


In [ ]:
wrong_sample = np.random.rand(1000)*2*std+mean-std
plt.figure(figsize=(10,6))
plt.hist(wrong_sample)
plt.tight_layout()
plt.show()

## Pasitikėjimo intervalai

Dabar apskaičiuokime pasitikėjimo intervalus beisbolo žaidėjų svoriams ir ūgiams. Naudosime kodą [iš šios stackoverflow diskusijos](https://stackoverflow.com/questions/15033511/compute-a-confidence-interval-from-sample-data):


In [ ]:
import scipy.stats

def mean_confidence_interval(data, confidence=0.95):
    a = 1.0 * np.array(data)
    n = len(a)
    m, se = np.mean(a), scipy.stats.sem(a)
    h = se * scipy.stats.t.ppf((1 + confidence) / 2., n-1)
    return m, h

for p in [0.85, 0.9, 0.95]:
    m, h = mean_confidence_interval(df['Weight'].fillna(method='pad'),p)
    print(f"p={p:.2f}, mean = {m:.2f} ± {h:.2f}")

## Hipotezių tikrinimas

Pažvelkime į skirtingas roles mūsų beisbolo žaidėjų duomenų rinkinyje:


In [ ]:
df.groupby('Role').agg({ 'Weight' : 'mean', 'Height' : 'mean', 'Age' : 'count'}).rename(columns={ 'Age' : 'Count'})

Patikrinkime hipotezę, kad pirmieji bazės žaidėjai yra aukštesni už antruosius bazės žaidėjus. Paprasčiausias būdas tai padaryti yra patikrinti pasitikėjimo intervalus:


In [ ]:
for p in [0.85,0.9,0.95]:
    m1, h1 = mean_confidence_interval(df.loc[df['Role']=='First_Baseman',['Height']],p)
    m2, h2 = mean_confidence_interval(df.loc[df['Role']=='Second_Baseman',['Height']],p)
    print(f'Conf={p:.2f}, 1st basemen height: {m1-h1[0]:.2f}..{m1+h1[0]:.2f}, 2nd basemen height: {m2-h2[0]:.2f}..{m2+h2[0]:.2f}')

Matome, kad intervalai nesikerta.

Statistiškai teisingesnis būdas patvirtinti hipotezę yra naudoti **Student t-testą**:


In [ ]:
from scipy.stats import ttest_ind

tval, pval = ttest_ind(df.loc[df['Role']=='First_Baseman',['Height']], df.loc[df['Role']=='Second_Baseman',['Height']],equal_var=False)
print(f"T-value = {tval[0]:.2f}\nP-value: {pval[0]}")

Funkcijos `ttest_ind` grąžinamos dvi reikšmės yra:
* p-reikšmė gali būti laikoma dviejų pasiskirstymų turinčių tokį pat vidurkį tikimybe. Mūsų atveju ji yra labai maža, reiškianti, kad yra stiprūs įrodymai, kad pirmieji žaidėjai yra aukštesni.
* t-reikšmė yra normalizuoto vidurkio skirtumo tarpinių reikšmių vertė, naudojama t-teste, ir ji lyginama su tam tikra slenkstine verte atitinkamam patikimumo lygiui.


## Normalaus pasiskirstymo simuliacija naudojant centrinę ribinę teoremą

Pseudoatsitiktinių skaičių generatorius Python sukurtas taip, kad duotų tolydžią pasiskirstymo funkciją. Jeigu norime sukurti normaliai pasiskirsčiusį generatorių, galime naudoti centrinę ribinę teoremą. Norėdami gauti normaliai pasiskirsčiusį reikšmę, tiesiog apskaičiuosime tolydžiai sugeneruoto imties vidurkį.


In [ ]:
def normal_random(sample_size=100):
    sample = [random.uniform(0,1) for _ in range(sample_size) ]
    return sum(sample)/sample_size

sample = [normal_random() for _ in range(100)]
plt.figure(figsize=(10,6))
plt.hist(sample)
plt.tight_layout()
plt.show()

## Koreliacija ir Blogoji Beisbolo Korporacija

Koreliacija leidžia rasti ryšius tarp duomenų sekų. Mūsų žaisliniame pavyzdyje įsivaizduokime, kad yra blogoji beisbolo korporacija, kuri žaidėjams moka pagal jų ūgį – kuo aukštesnis žaidėjas, tuo daugiau pinigų jis gauna. Tarkime, kad bazinis atlyginimas yra 1000 USD, o papildomas premijinis nuo 0 iki 100 USD priklauso nuo ūgio. Paimsime tikrus MLB žaidėjus ir apskaičiuosime jų įsivaizduojamus atlyginimus:


In [ ]:
heights = df['Height'].fillna(method='pad')
salaries = 1000+(heights-heights.min())/(heights.max()-heights.mean())*100
print(list(zip(heights, salaries))[:10])

Dabar apskaičiuokime tų sekų kovariaciją ir koreliaciją. `np.cov` suteiks mums taip vadinamą **kovariacinę matricą**, kuri yra kovariacijos išplėtimas keliems kintamiesiems. Kovariacinės matricos $M$ elementas $M_{ij}$ yra koreliacija tarp įvesties kintamųjų $X_i$ ir $X_j$, o įstrižainės reikšmės $M_{ii}$ yra $X_{i}$ dispersija. Panašiai, `np.corrcoef` suteiks mums **koreliacijos matricą**.


In [ ]:
print(f"Covariance matrix:\n{np.cov(heights, salaries)}")
print(f"Covariance = {np.cov(heights, salaries)[0,1]}")
print(f"Correlation = {np.corrcoef(heights, salaries)[0,1]}")

Koreliacija, lygi 1, reiškia, kad tarp dviejų kintamųjų yra stiprus **linijinis ryšys**. Linijinį ryšį galime vizualiai pamatyti pavaizdavę vieną reikšmę prieš kitą:


In [ ]:
plt.figure(figsize=(10,6))
plt.scatter(heights,salaries)
plt.tight_layout()
plt.show()

Pažiūrėkime, kas nutiks, jei ryšys nebus linijinis. Tarkime, kad mūsų korporacija nusprendė paslėpti akivaizdžią linijinę priklausomybę tarp ūgio ir atlyginimų, ir į formulę įtraukė tam tikrą nelinearumo elementą, pavyzdžiui, `sin`:


In [ ]:
salaries = 1000+np.sin((heights-heights.min())/(heights.max()-heights.mean()))*100
print(f"Correlation = {np.corrcoef(heights, salaries)[0,1]}")

Šiuo atveju koreliacija yra šiek tiek mažesnė, tačiau vis tiek gana didelė. Dabar, norėdami santykį padaryti dar mažiau akivaizdų, galime pridėti papildomo atsitiktinumo, pridėdami tam tikrą atsitiktinį kintamąjį prie atlyginimo. Pažiūrėkime, kas nutiks:


In [ ]:
salaries = 1000+np.sin((heights-heights.min())/(heights.max()-heights.mean()))*100+np.random.random(size=len(heights))*20-10
print(f"Correlation = {np.corrcoef(heights, salaries)[0,1]}")

In [ ]:
plt.figure(figsize=(10,6))
plt.scatter(heights, salaries)
plt.tight_layout()
plt.show()

> Ar galite atspėti, kodėl taškai išsirikiuoja į vertikalias linijas?

Mes pastebėjome koreliaciją tarp dirbtinai sukurto koncepto, kaip atlyginimas, ir stebimos kintamosios *ūgio*. Pažiūrėkime taip pat, ar dvi stebimos kintamosios, tokios kaip ūgis ir svoris, taip pat koreliuoja:


In [ ]:
np.corrcoef(df['Height'].ffill(),df['Weight'])

Deja, nepavyko gauti jokių rezultatų – tik keletą keistų `nan` reikšmių. Taip nutinka todėl, kad kai kurios mūsų serijos reikšmės nėra apibrėžtos, jas atitinka `nan`, o tai sukelia ir operacijos rezultatą būti neapibrėžtu. Pažvelgę į matricą matome, kad probleminis stulpelis yra `Weight`, nes atlikta savikoreliacija tarp `Height` reikšmių.

> Šis pavyzdys parodo **duomenų paruošimo** ir **valymo** svarbą. Be tinkamų duomenų negalime apskaičiuoti nieko.

Naudosime `fillna` metodą, kad užpildytume trūkstamas reikšmes, ir apskaičiuosime koreliaciją:


In [ ]:
np.corrcoef(df['Height'].fillna(method='pad'), df['Weight'])

Iš tiesų egzistuoja koreliacija, tačiau ji nėra tokia stipri kaip mūsų dirbtiniame pavyzdyje. Iš tiesų, jei pažvelgsime į taškų diagramą, kurioje vienas reikšmė lyginama su kita, ryšys būtų daug mažiau akivaizdus:


In [ ]:
plt.figure(figsize=(10,6))
plt.scatter(df['Weight'],df['Height'])
plt.xlabel('Weight')
plt.ylabel('Height')
plt.tight_layout()
plt.show()

## Išvada

Šiame užrašų knygelėje mes išmokome, kaip atlikti pagrindines operacijas su duomenimis, kad apskaičiuotume statistines funkcijas. Dabar žinome, kaip naudoti tvirtą matematikos ir statistikos aparatūrą, kad įrodytume kai kurias hipotezes, ir kaip apskaičiuoti pasitikėjimo intervalus bet kokiems kintamiesiems, turint duomenų imtį.


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**Atsakomybės limitas**:  
Šis dokumentas buvo išverstas naudojant dirbtinio intelekto vertimo paslaugą [Co-op Translator](https://github.com/Azure/co-op-translator). Nors siekiame tikslumo, prašome atkreipti dėmesį, kad automatizuoti vertimai gali turėti klaidų ar netikslumų. Originalus dokumentas gimtąja kalba laikomas autoritetingu šaltiniu. Kritinei informacijai rekomenduojamas profesionalus žmogaus atliktas vertimas. Mes neatsakome už jokią painiavą ar klaidingą interpretaciją, kylančią dėl šio vertimo naudojimo.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
